# Analyze the reproducibility of the simple DE 

We compute the coefficient of variation in $K = 30$ runs.


In [1]:
using Random, Printf, CSV, Tables
using Statistics
using DifferentialEvolution
Random.seed!(2);  

## Read data

In [2]:
const data_RT = CSV.File("data/RTFrance.csv") |> Tables.matrix;
const data_PW = CSV.File("data/Photowatt25.csv") |> Tables.matrix

const T_RT = 33 + 273.15  # temperature in Kalvin of the RTC France case
const T_PW = 45 + 273.15;  # temperature in Kalvin of the Photowatt case

# see Table 1 in our paper
const bounds_RT_sdm = Float64[0 1; 0 1; 1 2; 0 0.5; 0 100];
const bounds_RT_ddm = Float64[0 1; 0 1; 0 1; 1 2; 1 2; 0 0.5; 0 100]; 
const bounds_PW_sdm = Float64[0 2; 0 50; 1 50; 0 2; 0 2000]; 
const bounds_PW_ddm = Float64[0 2; 0 0.01; 0 50; 1 50; 1 50; 0 2; 0 2000];

In [3]:
include("models.jl") 

calculate_mae

In [4]:
const K = 30

# coefficient of variation
# each column denote one solution
function compute_cv(solutions)
    m = mean(solutions; dims=2)
    s = std(solutions; dims=2)
    return s ./ m
end

compute_cv (generic function with 1 method)

## SDM results

In [5]:
const np = 50  # number of individuals
const F = 0.6
const Cr = 0.9
const G_sdm = 800 # number of generations
const G_ddm = 1600 # number of generations
;

### RT

In [6]:
# given a parameter vector θ, compute its SSE as the fitness in DE
evaluator(θ) = calculate_sse(data_RT, (V, I) -> model_sdm(V, I, θ..., T_RT));

In [7]:
solutions = zeros(5, K)
# each run is new due to the involved randomness
for k = 1:K
    de = StdDE(np, bounds_RT_sdm, StdOptions(F, Cr); senses=-1)  # minimization
    evolve!(de, evaluator, G_sdm; stats=[])  
    best = best_individual(de)
    solutions[:, k] = best
end 

In [8]:
println("CV (SDM + RT):")
compute_cv(solutions)

CV (SDM + RT):


5×1 Matrix{Float64}:
 3.043252553794087e-10
 7.17267582257842e-8
 4.881845287107419e-9
 8.212350364377992e-9
 6.038057289051176e-8

### PW

In [9]:
# given a parameter vector θ, compute its SSE as the fitness in DE
# note that the temperature of the two datasets are different
evaluator(θ) = calculate_sse(data_PW, (V, I) -> model_sdm(V, I, θ..., T_PW));

In [10]:
solutions = zeros(5, K)
# each run is new due to the involved randomness
for k = 1:K
    de = StdDE(np, bounds_PW_sdm, StdOptions(F, Cr); senses=-1)  # minimization
    evolve!(de, evaluator, G_sdm; stats=[])
    best = best_individual(de)
    solutions[:, k] = best
end 

println("CV (SDM + PW):")
compute_cv(solutions)

CV (SDM + PW):


5×1 Matrix{Float64}:
 9.084875260855505e-10
 6.884770161741413e-8
 5.383295399127988e-9
 5.639590037704224e-9
 1.3280423032072743e-7

## DDM results

### RT

In [11]:
# given a parameter vector θ, compute its SSE as the fitness in DE
evaluator(θ) = calculate_sse(data_RT, (V, I) -> model_ddm(V, I, θ..., T_RT));

In [12]:
function exchange_if_small!(best, i, j)
    if best[i] > best[j]
        best[i], best[j] = best[j], best[i]
    end  
end

exchange_if_small! (generic function with 1 method)

In [13]:
solutions = zeros(7, K)
# each run is new due to the involved randomness
for k = 1:K
    de = StdDE(np, bounds_RT_ddm, StdOptions(F, Cr); senses=-1)  # minimization
    evolve!(de, evaluator, G_ddm; stats=[])
    best = best_individual(de)
    # assume I01 < I02, n1 < n2
    exchange_if_small!(best, 2, 3)
    exchange_if_small!(best, 4, 5)
    solutions[:, k] = best 
    # best_sse = best_fitness(de)
    # best_rmse = sqrt(best_sse / size(data_RT, 1))
    # @printf("%.5e\n", best_rmse)
end 

println("CV (DDM + RT):")
compute_cv(solutions)

CV (DDM + RT):


7×1 Matrix{Float64}:
 9.13809212402109e-7
 0.008271257187632492
 0.021181928238725896
 0.0004721585523995535
 1.7051992240779358e-5
 0.00024095923806942112
 0.0006470595136665226

### PW

In [14]:
evaluator(θ) = calculate_sse(data_PW, (V, I) -> model_ddm(V, I, θ..., T_PW));

In [15]:
solutions = zeros(7, K)
# each run is new due to the involved randomness
for k = 1:K
    de = StdDE(np, bounds_PW_ddm, StdOptions(F, Cr); senses=-1)  # minimization
    evolve!(de, evaluator, G_ddm; stats=[])
    best = best_individual(de)
    # assume I01 < I02, n1 < n2
    exchange_if_small!(best, 2, 3)
    exchange_if_small!(best, 4, 5)
    solutions[:, k] = best
    # best_sse = best_fitness(de)
    # best_rmse = sqrt(best_sse / size(data_PW, 1))
    # @printf("%.5e\n", best_rmse)
end 

println("CV (DDM + PW):")
compute_cv(solutions)

CV (DDM + PW):


7×1 Matrix{Float64}:
 4.748022351087864e-10
 0.6531759599288239
 0.0007898399070762047
 2.347371903208119e-7
 8.513312025611767e-6
 5.095420158688928e-9
 7.331491124315897e-8